## PINNs baseline (by DeepXDE)

In [1]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
import deepxde as dde
import numpy as np
print("NumPy version:", np.__version__)
# Backend pytorch
import torch

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


NumPy version: 1.26.4


In [2]:
# Problem parameter
c0 = 1.0 # initial concentration
D = 1.0 # solid-phase diffusion coefficient
R = 1.0
t = 1.0 # time 
It = 1.0 # current at time t
A = 1.0 # battery sheet area
L = 1.0 # thickness of positive/negative electrode
eps = 0.5 # solid phase volume fraction of each electrode
a = 3*eps/R # specific interfacial area
F = 1.0 # Faraday's constant
delta = It*R/(A*L*F*D*a*c0)

# x = (x, t)
def pde(x, y):
    # Most backends
    dy_x = dde.grad.jacobian(y, x, j=0)
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, j=0)
    # Backend pytorch
    return (
        dy_t
        - 2 * dy_x / (x[:, 0] + 1e-6)
        - dy_xx
    )

def boundary_l(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 0)

def boundary_r(x, on_boundary):
    return on_boundary and dde.utils.isclose(x[0], 1)

In [3]:
# This block considers the concentration from dataset
def func(x):
    return np.sin(np.pi * x[:, 0:1]) * np.exp(-x[:, 1:])

observe_x = np.vstack((np.linspace(-1, 1, num=10), np.random.randn(10) * 0.1)).T
observe_y = dde.icbc.PointSetBC(observe_x, func(observe_x), component=0)

print(func(observe_x))

[[-1.27682078e-16]
 [-5.97464273e-01]
 [-1.00999942e+00]
 [-9.57114311e-01]
 [-3.14451504e-01]
 [ 3.83256928e-01]
 [ 8.97891653e-01]
 [ 1.02048904e+00]
 [ 6.62673725e-01]
 [ 1.25230861e-16]]


In [ ]:
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, delta)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc_l = dde.icbc.NeumannBC(geomtime, lambda X: 0.0, boundary_l)
bc_r = dde.icbc.NeumannBC(geomtime, lambda X: delta, boundary_r)

ic = dde.icbc.IC(geomtime, lambda X: 1.0, lambda _, on_initial: on_initial)
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_l, bc_r, ic, observe_y],
    num_domain=40,
    num_boundary=20,
    num_initial=10,
    anchors=observe_x,
    num_test=10000,
)

layer_size = [2] + [32] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)

model.compile("adam", lr=0.001)
losshistory, train_state = model.train(iterations=20000)

dde.saveplot(losshistory, train_state, issave=False, isplot=True)

Compiling model...
'compile' took 0.545062 s

Training model...

Step      Train loss                                            Test loss                                             Test metric
0         [4.88e+10, 1.04e-01, 2.75e-01, 7.56e-01, 3.48e-01]    [2.72e+08, 1.04e-01, 2.75e-01, 7.56e-01, 3.48e-01]    []  
1000      [1.03e+05, 6.90e-08, 4.45e-01, 2.67e-01, 7.08e-01]    [3.85e+02, 6.90e-08, 4.45e-01, 2.67e-01, 7.08e-01]    []  
2000      [6.04e+04, 1.67e-08, 4.45e-01, 2.48e-01, 7.27e-01]    [1.69e+02, 1.67e-08, 4.45e-01, 2.48e-01, 7.27e-01]    []  
